In [11]:
#FST's


class State:
    def __init__(self):
        self.transitions = {}  # char -> next State
        self.output = None     # suggestion or associated data
        self.is_final = False

class FST:
    def __init__(self):
        self.start = State()

    def insert(self, word, output=None):
        node = self.start
        for char in word:
            if char not in node.transitions:
                node.transitions[char] = State()
            node = node.transitions[char]
        node.is_final = True
        node.output = output if output else word

    def autocomplete(self, prefix):
        node = self.start
        for char in prefix:
            if char not in node.transitions:
                return []
            node = node.transitions[char]
        return self._collect_outputs(node)

    def _collect_outputs(self, node):
        results = []
        stack = [(node, "")]
        while stack:
            current, path = stack.pop()
            if current.is_final:
                results.append(current.output)
            for char, next_node in current.transitions.items():
                stack.append((next_node, path + char))
        return results


In [12]:
fst = FST()
fst.insert("hello")
fst.insert("help")
fst.insert("helium")
fst.insert("hero")
fst.insert("man")
fst.insert("genre")
fst.insert("gentle")
fst.insert("gem")

print(fst.autocomplete("ge"))   # Output: ['gem', 'gentle', 'genre']
print(fst.autocomplete("he"))   # Output: ['hero','helium', 'help', 'hello']
print(fst.autocomplete("hel"))   # Output: ['hello', 'help', 'helium']



['gem', 'gentle', 'genre']
['hero', 'helium', 'help', 'hello']
['helium', 'help', 'hello']


In [9]:
#Weighted FST's

class State:
    def __init__(self):
        self.transitions = {}  # char -> next State
        self.output = None     # suggestion or associated data
        self.is_final = False
        self.weight = float('inf')  # Lower weight = higher priority

class WeightedFST:
    def __init__(self):
        self.start = State()

    def insert(self, word, output=None, weight=1):
        node = self.start
        for char in word:
            if char not in node.transitions:
                node.transitions[char] = State()
            node = node.transitions[char]
        node.is_final = True
        node.output = output if output else word
        node.weight = weight

    def autocomplete(self, prefix):
        node = self.start
        for char in prefix:
            if char not in node.transitions:
                return []
            node = node.transitions[char]
        results = self._collect_outputs(node)
        # Sort by weight ascending (lower weight = better)
        results.sort(key=lambda x: x[1])
        return [output for output, weight in results]

    def _collect_outputs(self, node):
        results = []
        stack = [(node, "")]
        while stack:
            current, path = stack.pop()
            if current.is_final:
                results.append((current.output, current.weight))
            for char, next_node in current.transitions.items():
                stack.append((next_node, path + char))
        return results



In [10]:
# Example usage
wfst = WeightedFST()
wfst.insert("hello", weight=1)    # Most popular
wfst.insert("help", weight=2)
wfst.insert("helium", weight=3)
wfst.insert("hero", weight=4)
wfst.insert("man", weight=10)
wfst.insert("genre", weight=8)
wfst.insert("gentle", weight=6)
wfst.insert("gem", weight=5)

print(wfst.autocomplete("ge"))   # ['gem', 'gentle', 'genre']
print(wfst.autocomplete("he"))   # ['hello', 'help', 'helium', 'hero']
print(wfst.autocomplete("hel"))  # ['hello', 'help', 'helium']


['gem', 'gentle', 'genre']
['hello', 'help', 'helium', 'hero']
['hello', 'help', 'helium']
